In [ ]:
from diffusion_conditional_masked import *
from train import test_acc, source_domain_numpy, domain_test_numpy
import matplotlib.pyplot as plt

In [ ]:
# hardcoding these here
seed = 0

n_epoch = 100
batch_size = 256
n_T = 400 # 500
device = "cuda:2"
n_classes = 10 + 1
target_domain_index = 1

n_domains = 4
n_feat = 128 # 128 ok, 256 better (but slower)
lrate = 1e-4
save_model = True
save_dir = './data/diffusion_outputs_masked_Apr21/'
ws_test = [0.0, 0.5, 2.0] # strength of generative guidance
save_gif = False

in_channels = 3 #1

ddpm = DDPM(nn_model=ContextUnet(in_channels=in_channels, n_feat=n_feat, n_classes=n_classes, n_domains=n_domains), betas=(1e-4, 0.02), n_T=n_T, device=device, drop_prob=0.1)
ddpm.to(device)

ddpm.load_state_dict(torch.load('data/diffusion_outputs_masked/model_99.pth',map_location=device))

In [ ]:
config = {'base_root':'../data', 'source_dataset':'mnist,svhn,syn_digits', 'target_dataset':'mnist', 'batch_size':32,\
             'resolution':28,'num_workers':4, 'seed':1, 'num_epochs': 100}

#transforms_test = transforms.Compose([transforms.Resize(config['resolution']),transforms.ToTensor(),transforms.Normalize([0.5], [0.5])])
transforms_test = transforms.Compose([transforms.Resize(config['resolution']),transforms.ToTensor()])

test_set = domain_test_numpy(root= config['base_root'], root_t=config['target_dataset'], transform=transforms_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=config['batch_size'], shuffle=False, num_workers=config['num_workers'], pin_memory=True, worker_init_fn=np.random.seed, drop_last=True)




In [ ]:
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # calculate outputs by running images through the network
        #outputs = model(images)
        break

In [ ]:
with torch.no_grad():
    #acc = torch.zeros(len(images) * 11).to(device)
    for sample in range(1000):
        #print(sample)
        #pseudo_labels = torch.ones(len(images), dtype=int) * try_label
        #domains = torch.ones(len(images), dtype=int) * 0
        
        #images, pseudo_labels, domains = images.to(device), pseudo_labels.to(device), domains.to(device)
        images = images.to(device)
        out = ddpm(images, labels.to(device), torch.ones(len(images), dtype=int).to(device) * 0)
        #loss = ((out[0] - out[1])**2).sum(dim=1).sum(dim=1).sum(dim=1)
        #loss = ddpm.loss_mse(out[0], out[1])
        print(out.mean())
        #acc += loss

In [ ]:
dic = {}
#for try_label in range(10):
    #print(try_label)
ddpm.drop_prob = 0.0
with torch.no_grad():
    acc = torch.zeros(len(images) * 11).to(device)
    for sample in tqdm(range(1000)):
        #print(sample)
        #pseudo_labels = torch.ones(len(images), dtype=int) * try_label
        #domains = torch.ones(len(images), dtype=int) * 0
        
        #images, pseudo_labels, domains = images.to(device), pseudo_labels.to(device), domains.to(device)
        images = images.to(device)
        out = ddpm.forward_without_loss(images, 0)
        loss = ((out[0] - out[1])**2).sum(dim=1).sum(dim=1).sum(dim=1)
        #loss = ddpm.loss_mse(out[0], out[1])
        #print(loss)
        acc += loss

In [ ]:
correct = 0
for i in range(32):
    min_index = 0
    min_val = acc[i* 10 ]
    for try_label in range(10):
        if(acc[i* 11 + try_label] < min_val):
            min_val = acc[i* 11 + try_label]
            min_index = try_label
    correct += (min_index == int(labels[i]))
    #print(min_index, labels[i])
correct

In [ ]:
with torch.no_grad():
    x_gen, _ = ddpm.sample(11 * 4, (3, 28, 28), device, guide_w = 2)

In [ ]:
index = 30
img = x_gen[index]
img = (img - img.min())/(img.max() - img.min())
plt.imshow(img.permute(1,2,0).cpu())

In [ ]:
import matplotlib.pyplot as plt
index = 16
print(labels[index])
plt.imshow(images[index].permute(1,2,0).cpu())

In [ ]:
acc[11:22]

In [ ]:
#self.n_classes
x = images
n_classes = 10
domain = 0

images.shape
c = torch.arange(n_classes, dtype=int).repeat(len(x))
d = torch.ones(len(c), dtype=int) * domain
noise = torch.randn_like(x) 

In [ ]:
_ts = torch.randint(1, self.n_T+1, (x.shape[0],)).to(self.device) 
_ts.shape

In [ ]:
torch.arange(n_classes).repeat_interleave(2)

In [ ]:
noise = noise.repeat_interleave(n_classes)
x = x.repeat_interleave(n_classes)

In [ ]:
dic = {}
for try_label in range(10):
    print(try_label)
    with torch.no_grad():
        dic[try_label] = torch.zeros(len(images)).to(device)
        for sample in range(100):
            pseudo_labels = torch.ones(len(images), dtype=int) * try_label
            domains = torch.ones(len(images), dtype=int) * 0
            
            images, pseudo_labels, domains = images.to(device), pseudo_labels.to(device), domains.to(device)
            out = ddpm.forward_without_loss(images, pseudo_labels, domains)
            loss = ((out[0] - out[1])**2).sum(dim=1).sum(dim=1).sum(dim=1)
            dic[try_label] += loss

In [ ]:
for i in range(256):
    min_index = 0
    min_val = dic[0][i]
    for try_label in range(10):
        if(dic[try_label][i] < min_val):
            min_val = dic[try_label][i]
            min_index = try_label
    print(min_index, labels[i])

In [ ]:
for i in range(10):
    print(dic[i][0])

In [ ]:
loss[labels == 2].mean()

In [ ]:
domains